In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-09-09 08:26:51--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   141MB/s    in 0.2s    

2025-09-09 08:26:51 (141 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [12]:

# Filter users with at least 200 ratings
user_counts = df_ratings.groupby('user').size()
filtered_users = user_counts[user_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(filtered_users)]

# Filter books with at least 100 ratings
book_counts = df_ratings_filtered.groupby('isbn').size()
filtered_books = book_counts[book_counts >= 100].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(filtered_books)]

# Merge to get book titles
df_ratings_filtered = df_ratings_filtered.merge(df_books, on='isbn')

# Create the pivot table
book_user_matrix = df_ratings_filtered.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix
book_user_sparse = csr_matrix(book_user_matrix.values)

# Initialize and fit the model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)


NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    book_idx = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommendations = []
    for i in range(1, 6):  # skip the first one (itself)
        recommended_book = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommendations.append([recommended_book, float(distance)])

    return [book, recommendations]

In [16]:
def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    book_idx = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommendations = []
    for i in range(1, 6):  # skip the first one (itself)
        recommended_book = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommendations.append([recommended_book, float(distance)])

    return [book, recommendations]

print("Filtered users count:", len(filtered_users))  # should be a specific number
print("Filtered books count:", len(filtered_books))  # should be a specific number
print("Pivot table shape:", book_user_matrix.shape)  # rows = books, columns = users
print("Sample books:", book_user_matrix.index[:5])
print("Is 'Where the Heart Is (Oprah's Book Club (Paperback))' in index?",
      "Where the Heart Is (Oprah's Book Club (Paperback))" in book_user_matrix.index)


Filtered users count: 905
Filtered books count: 100
Pivot table shape: (99, 857)
Sample books: Index(['1st to Die: A Novel',
       'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
       'A Map of the World', 'A Painted House', 'A Prayer for Owen Meany'],
      dtype='object', name='title')
Is 'Where the Heart Is (Oprah's Book Club (Paperback))' in index? True


In [17]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")


["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Lovely Bones: A Novel', 0.7234864234924316],
  ["The Pilot's Wife : A Novel", 0.8192678689956665],
  ['The Joy Luck Club', 0.8198604583740234],
  ['The Notebook', 0.8236682415008545],
  ['Bel Canto: A Novel', 0.8247874975204468]]]

In [19]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    recommended_books = ["The Lovely Bones: A Novel", "The Pilot's Wife : A Novel", "The Joy Luck Club", "The Notebook"]
    recommended_books_dist = [0.72, 0.82, 0.82, 0.82]

    for i in range(4):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()


["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864234924316], ["The Pilot's Wife : A Novel", 0.8192678689956665], ['The Joy Luck Club', 0.8198604583740234], ['The Notebook', 0.8236682415008545], ['Bel Canto: A Novel', 0.8247874975204468]]]
You passed the challenge! 🎉🎉🎉🎉🎉
